# COPA DO MUNDO 2022 - QATAR!
- utilizando Python 3.8.5

# Bibliotecas necessárias

In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
# verificando a versão do pandas
print(pd.__version__)

1.4.3


# Importando os dados (fontes de dados)
- já tratadas

In [6]:
# Grupos da Copa 2022
dict_table = pickle.load(open('data/dict_table','rb'))

In [5]:
# Históricos de jogos das copas 
#   HomeGoals  - Gols do time "mandante"
#   AwayGoals  - Gols do time "visitante"
#   Totalgoals - Somatótio
df_historical_data = pd.read_csv('data/clean_fifa_worldcup_matches.csv')

# são as partidas da copa
df_fixture = pd.read_csv('data/clean_fifa_worldcup_fixture.csv')

In [26]:
df_historical_data

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


# Função para predizer pontos
- No caso, são considerados os números de gols!
    - Usando distribuição Poisson

Por que? Bem, vamos dar uma olhada na definição da distribuição de Poisson.

A distribuição de Poisson é uma distribuição de probabilidade discreta que descreve o número de eventos que ocorrem em um intervalo de tempo fixo ou região de oportunidade.

Se pensarmos em um gol como um evento que pode acontecer nos 90 minutos de uma partida de futebol, poderíamos calcular a probabilidade do número de gols que poderiam ser marcados em uma partida pela Equipe A e pela Equipe B.

Mas isso não é suficiente. Ainda precisamos atender às suposições da distribuição de Poisson.

1. O número de eventos pode ser contado (uma partida pode ter 1, 2, 3 ou mais gols)
2. A ocorrência de eventos é independente (a ocorrência de um objetivo não deve afetar a probabilidade de outro objetivo)
3. A taxa na qual os eventos ocorrem é constante (a probabilidade de um objetivo ocorrer em um determinado intervalo de tempo deve ser exatamente a mesma para todos os outros intervalos de tempo do mesmo comprimento)
4. Dois eventos não podem ocorrer exatamente no mesmo instante no tempo (dois objetivos não podem ocorrer ao mesmo tempo)

Sem dúvida, as suposições 1 e 4 são atendidas, mas 2 e 3 são parcialmente verdadeiras. Dito isto, vamos supor que as suposições 2 e 3 são sempre verdadeiras.



## Para fazer as previsões:

**lambda**: mediana de gols em 90 minutos (Equipe A e Equipe B)

**x**: número de gols em uma partida que poderiam ser marcados pela Equipe A e Equipe B (0 a 10)

Para calcular o **lambda**, precisamos da média de gols marcados/sofridos por cada seleção nacional, vejamos abaixo:

### Calculando a "força" do time (Lambda)
- gols marcados
- gols sofridos

In [7]:
# Com base no histórico, para cada seleção, mostra a quantidade de gols que fez e a quantidade de gols que levou
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


### Calculando a Previsão

Em português claro, calcula quantos pontos as equipes de casa (mandantes) e fora de casa (visitantes) receberiam. Para isso, foi calculado **lambda** para cada equipe usando a fórmula:

*predict_points = average_goals_scored * average_goals_conceded*

Então foi simulado todas as pontuações possíveis de uma partida de 0-0 a 10-10. Uma vez que eu tenho lambda e x, eu uso a fórmula da distribuição de Poisson para calcular.p

Acumula o valor de, digamos, a partida terminar em 1-0 (vitórias em casa), 1-1 (empate) ou 0-1 (vitórias fora), respectivamente. Finalmente, os pontos são calculados com a fórmula abaixo.
- prob_home (vitoria time "mandante")
- prob_draw (empate)
- prob_away (vitoria time "visitante")
- p

In [8]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

### Testando a função

In [43]:
dict_table['Group B']

,Team,Pts
0,England,12.0
1,Wales,10.0
2,United States,6.0
3,Iran,4.0


In [9]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(2.2356147635326007, 0.5922397535606193)
(2.3129151525530505, 0.5378377125059863)
(0, 0)


Isso significa que a Inglaterra obteria 2,23 pontos, enquanto os EUA receberiam 0,59. Decimais porque estou usando probabilidades.

Se aplicarmos esta função a todos os jogos da fase de grupos, obteremos a 1ª e a 2ª posição de cada grupo, daí os seguintes jogos nos oitavos-de-final.

In [47]:
# tabela de jogos: https://www.uol.com.br/esporte/futebol/copa-do-mundo/tabela-da-copa/grupos/grupo-a

# primeiros 4 jogos...
print('Qatar (H) vs Ecuador')
print(predict_points('Qatar (H)', 'Ecuador'))   # Equador ganhou (não OK)

print('England vs Iran')
print(predict_points('England', 'Iran'))        # Inglaterra ganhou (OK)

print('Senegal vs Netherlands')
print(predict_points('Senegal', 'Netherlands')) # 

print('United States vs Wales')
print(predict_points('United States', 'Wales')) #

Qatar (H) vs Ecuador
(0, 0)
England vs Iran
(2.388251010951943, 0.43330539903202214)
Senegal vs Netherlands
(0.8966936039774208, 1.8974813628251535)
United States vs Wales
(0.9356595028112644, 1.8033715394312302)


# Prevendo a World Cup

## Fase de Grupos
- apliando a função explicada acima

In [29]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [30]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

### Classificados
- Lembrando que é uma previsão...

- Grupo A
    - Netherlands
    - Senegal
- Grupo B
    - England
    - Wales
- Grupo C
    - Argentina
    - Poland
- Grupo D
    - France
    - Denmark
- Grupo E
    - Germany
    - Spain
- Grupo F
    - Croatia
    - Belgium
- Grupo G
    - Brazil
    - Switzerland
- Grupo H
    - Portugal
    - Uruguay

## Resumindo 

In [13]:
# sequência de próximos jogos (com bases nas primeiras colocações)
df_fixture_knockout

,home,score,away,year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [14]:
# Montando a tabela do Mata a Mata
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Senegal,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


## Prevendo os Mata-Matas
Para os mata-matas, não preciso prever os pontos, mas o vencedor de cada chave. É por isso que a função com base na função anterior.

*get_winnerpredict_points*

In [33]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

Simplificando, se o vencedor for maior do que o vencedor é o time da casa (mandante), caso contrário, o vencedor é o time visitante.
- *points_home*
- *points_away*

Graças à função, posso obter os resultados dos colchetes anteriores.
- *get_winner*

In [16]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


## Quartas de Finais
- Se usar o novamente a mesma lógica, posso prever o vencedor das Quartas!
    - get_winner

In [17]:
# Atualizar com base na programação dos jogos (monta a tabela de jogos das Quartas de Finais)
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [18]:
# Montando a tabela de jogos das Quartas de Finais
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Spain,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [19]:
# Prevendo o vencedor desta fase
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


## Semifinal
- Se usar o novamente a mesma lógica, posso prever o Resultado das Semi-Finais.
    - get_winner

In [20]:
# Atualizar com base na programação dos jogos (monta a tabela de jogos das Semifinais)
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [21]:
# Prevendo o vencedor desta fase
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


## Final
- Se usar o novamente a mesma lógica, posso prever o Resultado da Final.
    - get_winner

In [22]:
# Atualizar com base na programação dos jogos (monta a tabela de jogos da Final)
update_table(df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [23]:
# Prevendo o vencedor desta fase
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil


Deu **BRASIL** nas previsões, vamos ver na real... 

Este foi um excelente estudo do Frank Andrade, disponível em: https://towardsdatascience.com/predicting-the-fifa-world-cup-2022-with-a-simple-model-using-python-6b34bdd4f2a5

Aqui foi apenas uma tradução, conforme meu entendimento, da solução feita pelo Frank Andrade.